In [ ]:
#%load_ext autoreload
#%autoreload 2

In [1]:
#carga de datasets
from utils.DatasetStorage import Dataset
from utils.paths import *

#clasificadores
from utils.clasificacion import *

#adaptacion
from utils.adaptacion import msda_theano_pseudo_grid_search 

#otros
import os
import numpy as np
import pandas as pd
from sklearn.externals import joblib
import itertools

#variables para guardar los resultados
tipo = pruebas[1]
dataset_name = datasets[0]

Using Theano backend.
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


# Adaptación


## Creación de modelos de adaptación.

Para cada dominio se entrenan distintos modelos según los parámetros enviados.

Cada modelo es guardado en la ruta: models/amazon/msda/me1\_[dominio\_objetivo]_[numero_de_modelo].pkl

Todas las rutas son guardadas en una lista, la cual es almacenada en: models/amazon/msda/me1_models_paths.pkl

```python
paths_list = [ruta1, ..., ruta_n]
```

## Busqueda del mejor modelo por dominio

Por cada dominio se busca el mejor modelo de adaptación.

Esto se obtiene adaptando los datos de entrenamiento de cada dominio por cada modelo creado y realizando Grid-Search y Cross-Validation con estos datos.

El modelo que logre un mejor valor de Cross-Validation es considerado el mejor modelo para adaptar y queda guardado en un diccionario de la forma:

```python
best_models = {
    [dominio_1] = ruta,
    .
    .
    .
    [dominio_n] = ruta,
}
```

Este diccionario queda almacenado en la ruta:
    models/amazon/msda/me1_best_models.pkl

# Pruebas con el dataset Amazon (3000 Dimensiones)

In [2]:
dims = 3000

In [3]:
print tipo
print dataset_name
print dims
print data_path

msda
amazon
3000
data


In [4]:
# cargando dataset Amazon
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
domains = dataset_object.domains

Dataset already splitted


In [6]:
# se obtienen todos los valores X disponibles para realizar adaptacion
X = dataset_object.get_all_X()
X = np.asarray(X[: , :dims])

print X.shape

print "Todos los datos disponibles obtenidos"

(26077, 3000)
Todos los datos disponibles obtenidos


In [7]:
models_paths = os.path.join(models_path, dataset_name, tipo, 'me1_%d_models_paths.pkl' % dims)
paths_list = []

# si existe el archivo con las rutas
# se carga la lista con las rutas
if os.path.exists(models_paths):
    print "Cargando rutas de modelos adaptados."
    paths_list = joblib.load(models_paths)
    print "Rutas cargadas en la variable 'paths_list'"
# si no
# se entrenan los modelos y se obtiene la lista con rutas
else:
    #se establecen los parametros para los modelos
    parametros = {
        'noises': [0.3, 0.5, 0.8],
        'layers': [1, 3, 5]
    }

    print "Creando modelos de adaptacion..."
    
    folder_path = os.path.join(models_path, dataset_name, tipo)
    prefix = "me1_%d_" % dims
        
    paths_list = msda_theano_pseudo_grid_search(X, parametros, folder_path, prefix)

    print "\nCreacion de modelos terminada\nGuardando rutas en %s" % models_paths
    joblib.dump(paths_list, models_paths)
    print "Rutas cargadas en la variable 'paths_list'"

Creando modelos de adaptacion...
	pr: 0.300 - l: 1
	Entrenando capa  1
	Entrenado en 2.37m

	Guardando modelo en models/amazon/msda/me1_3000_0.pkl
	pr: 0.300 - l: 3
	Entrenando capa  1
	Entrenando capa  2
	Entrenando capa  3
	Entrenado en 7.07m

	Guardando modelo en models/amazon/msda/me1_3000_1.pkl
	pr: 0.300 - l: 5
	Entrenando capa  1
	Entrenando capa  2
	Entrenando capa  3
	Entrenando capa  4
	Entrenando capa  5
	Entrenado en 11.82m

	Guardando modelo en models/amazon/msda/me1_3000_2.pkl
	pr: 0.500 - l: 1
	Entrenando capa  1
	Entrenado en 2.37m

	Guardando modelo en models/amazon/msda/me1_3000_3.pkl
	pr: 0.500 - l: 3
	Entrenando capa  1
	Entrenando capa  2
	Entrenando capa  3
	Entrenado en 7.10m

	Guardando modelo en models/amazon/msda/me1_3000_4.pkl
	pr: 0.500 - l: 5
	Entrenando capa  1
	Entrenando capa  2
	Entrenando capa  3
	Entrenando capa  4
	Entrenando capa  5
	Entrenado en 11.87m

	Guardando modelo en models/amazon/msda/me1_3000_5.pkl
	pr: 0.800 - l: 1
	Entrenando capa  1
	En

In [8]:
print "Modelos almacenados en:"
for ruta in paths_list:
    print "\t", ruta

Modelos almacenados en:
	models/amazon/msda/me1_3000_0.pkl
	models/amazon/msda/me1_3000_1.pkl
	models/amazon/msda/me1_3000_2.pkl
	models/amazon/msda/me1_3000_3.pkl
	models/amazon/msda/me1_3000_4.pkl
	models/amazon/msda/me1_3000_5.pkl
	models/amazon/msda/me1_3000_6.pkl
	models/amazon/msda/me1_3000_7.pkl
	models/amazon/msda/me1_3000_8.pkl


In [9]:
best_models_paths = os.path.join(models_path, dataset_name, tipo, "me1_%d_best_models.pkl" % dims)
best_models = {}

if os.path.exists(best_models_paths):
    print "Cargando rutas de los mejores modelos..."
    best_models = joblib.load(best_models_paths)
    print "Rutas cargadas"  
else:
    print "Obteniendo mejores modelos..."
    for domain in domains:
        print domain
        i = 1
        best_score = 0
        best_path = None
        best_model = None
        
        for msda_model_path in paths_list:
            
            msda_model = joblib.load(msda_model_path)
            theano_model = msda_model['model']
            pr = msda_model['pr']
            n_layers = msda_model['l']

            print "\t%d) l=%d, pr=%.3f " % (i, msda_model['l'], msda_model['pr']),

            # se obtienen los datos del dominio
            X_tr = labeled[domain]['X_tr'][:, :dims].todense()
            y_tr = np.asarray(labeled[domain]['y_tr'].todense()).argmax(axis=1)

            # se adaptan los datos
            X_deep = theano_model.predict(X_tr)

            # se obtiene el mejor score con GridSearch
            new_clf = get_best_score(X_deep, y_tr, classifier='SVC', n_jobs=4)
            new_score = new_clf.best_score_
            print "score: %.4f" % new_score
            # se guarda si es el mejor para el modelo
            if new_score > best_score:
                best_score = new_score
                best_path = msda_model_path
                best_model = msda_model
            i = i+1

        #se guarda el mejor modelo para este dominio
        print "Mejor modelo: l=%d, pr=%.3f" % (best_model['l'], best_model['pr'])
        best_models[domain] = best_path
        
    
    # se guarda el diccionario con las mejores rutas
    joblib.dump(best_models, best_models_paths)
    print "Rutas guardadas en ", best_models_paths

Obteniendo mejores modelos...
electronics
	1) l=1, pr=0.300  score: 0.9045
	2) l=3, pr=0.300 

/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with Standar

 score: 0.9001
	3) l=5, pr=0.300  score: 0.8932
	4) l=1, pr=0.500  score: 0.9054
	5) l=3, pr=0.500  score: 0.8847
	6) l=5, pr=0.500  score: 0.8053
	7) l=1, pr=0.800  score: 0.9107
	8) l=3, pr=0.800  score: 0.8006
	9) l=5, pr=0.800  score: 0.7988
Mejor modelo: l=1, pr=0.800
dvd
	1) l=1, pr=0.300  score: 0.8481
	2) l=3, pr=0.300  score: 0.8462
	3) l=5, pr=0.300  score: 0.8139
	4) l=1, pr=0.500  score: 0.8467
	5) l=3, pr=0.500  score: 0.7924
	6) l=5, pr=0.500  score: 0.7244
	7) l=1, pr=0.800  score: 0.8625
	8) l=3, pr=0.800  score: 0.7311
	9) l=5, pr=0.800  score: 0.6917
Mejor modelo: l=1, pr=0.800
kitchen
	1) l=1, pr=0.300  score: 0.9204
	2) l=3, pr=0.300  score: 0.9107
	3) l=5, pr=0.300  score: 0.8908
	4) l=1, pr=0.500  score: 0.9196
	5) l=3, pr=0.500  score: 0.8820
	6) l=5, pr=0.500  score: 0.8072
	7) l=1, pr=0.800  score: 0.9213
	8) l=3, pr=0.800  score: 0.8183
	9) l=5, pr=0.800  score: 0.8227
Mejor modelo: l=1, pr=0.800
books
	1) l=1, pr=0.300  score: 0.8402
	2) l=3, pr=0.300  score:

In [10]:
for dominio, ruta in best_models.items():
    print "%s - %s" % (dominio, ruta)

dvd - models/amazon/msda/me1_3000_6.pkl
electronics - models/amazon/msda/me1_3000_6.pkl
books - models/amazon/msda/me1_3000_1.pkl
kitchen - models/amazon/msda/me1_3000_6.pkl


In [11]:
#diccionario para mantener los dominios adaptados
adapted = {}

i = 0
for domain in domains:
    best_model_path = best_models[domain]
    
    if os.path.exists(best_model_path):
        print "Cargando mejor modelo para %s" % domain
        msda_model = joblib.load(best_model_path)
        theano_model = msda_model['model']
        
        print "\tAdaptando dominio"
        X_tr = labeled[domain]['X_tr'][:, :dims].todense()

        tr_reps = theano_model.predict(X_tr)

        adapted[domain] = {
            'X_tr': tr_reps
        }
        
        i = i+1
    else:
        print "\tGenerar mejor modelo para %s" % domain
        
print "%d/%d dominios adaptados" % (i, len(domains))

Cargando mejor modelo para electronics
	Adaptando dominio
Cargando mejor modelo para dvd
	Adaptando dominio
Cargando mejor modelo para kitchen
	Adaptando dominio
Cargando mejor modelo para books
	Adaptando dominio
4/4 dominios adaptados


## Pruebas de clasificación

In [12]:
df = pd.DataFrame(columns=dataframe_columns)

i=0
tareas = len(domains)*(len(domains)-1)
pairs = list(itertools.permutations(domains, 2))

# por cada par posible para adaptar
for src, tgt in pairs:
    #se carga el mejor modelo para el dominio fuente
    ruta = best_models[src]
    modelo = joblib.load(ruta)
    msda_adapter = modelo['model']

    print "Tarea %d de %d" % (i+1, tareas)

    #baseline in-domain error
    #e_b(T,T)
    #entrenado en dominio tgt y probado en dominio tgt
    X_tr = labeled[tgt]['X_tr'][:, :dims].todense()
    y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)

    X_ts = labeled[tgt]['X_ts'][:, :dims].todense()
    y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

    # se crean las rutas para cargar o crear los modelos
    model_name = "%d_%s.pkl" % (dims, tgt)
    model_path = os.path.join(models_path, dataset_name, "indomain", model_name)


    #Se realiza una clasificacion, estimando los parametros mediante cv
    svc = load_best_score(model_path, X_tr, y_tr)
    b_error = 1-svc.score(X_ts, y_ts)


    #############
    #### mSDA ###
    #############
    print "Adaptando dominios..."

    #entrenado en dominio src y probado en dominio tgt adaptados
    X_tr_a = adapted[src]['X_tr']
    y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)

    X_ts_a = msda_adapter.predict(X_ts)
    y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

    # se calcula el transfer error
    svc_a = get_best_score(X_tr_a, y_tr, classifier='SVC', n_jobs=4)
    t_error = 1-svc_a.score(X_ts_a, y_ts)

    # transfer loss t
    # t_error - b_error
    t_loss = t_error - b_error

    tarea = src[0]+'->'+tgt[0]
    df.loc[i] = ['mSDA',tarea,src,tgt,b_error*100,t_error*100, t_loss*100]

    i+=1
    
print "\nPruebas completadas"

Tarea 1 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 2 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 3 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 4 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 5 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 6 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 7 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 8 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 9 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 10 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 11 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 12 de 12
Cargando modelo existente.
Adaptando dominios...

Pruebas completadas


In [13]:
df

,Adaptacion,Tarea,Fuente,Objetivo,Baseline error,Transfer error,Transfer loss
0,mSDA,e->d,electronics,dvd,15.762894,25.670642,9.907748
1,mSDA,e->k,electronics,kitchen,8.587715,9.610240,1.022526
2,mSDA,e->b,electronics,books,15.157879,25.920648,10.762769
3,mSDA,d->e,dvd,electronics,11.127778,24.348109,13.220331
4,mSDA,d->k,dvd,kitchen,8.587715,18.142954,9.555239
5,mSDA,d->b,dvd,books,15.157879,21.900548,6.742669
6,mSDA,k->e,kitchen,electronics,11.127778,12.200305,1.072527
7,mSDA,k->d,kitchen,dvd,15.762894,25.378134,9.615240
8,mSDA,k->b,kitchen,books,15.157879,29.000725,13.842846
9,mSDA,b->e,books,electronics,11.127778,23.143079,12.015300


In [14]:
new_scores_path = os.path.join(scores_path,dataset_name, tipo, "me1_%d.csv" % (dims))

print "Guardando en %s" % new_scores_path
df.to_csv(new_scores_path, columns=df.columns)
print "Resultados guardados."

Guardando en scores/amazon/msda/me1_3000.csv
Resultados guardados.


# Pruebas con el dataset Amazon (1000 Dimensiones)

In [15]:
dims = 1000

In [16]:
print tipo
print dataset_name
print dims
print data_path

msda
amazon
1000
data


In [17]:
# cargando dataset Amazon
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
domains = dataset_object.domains

Dataset already splitted


In [18]:
# se obtienen todos los valores X disponibles para realizar adaptacion
X = dataset_object.get_all_X()
X = np.asarray(X[: , :dims])

print X.shape

print "Todos los datos disponibles obtenidos"

(26077, 1000)
Todos los datos disponibles obtenidos


In [19]:
models_paths = os.path.join(models_path, dataset_name, tipo, 'me1_%d_models_paths.pkl' % dims)
paths_list = []

# si existe el archivo con las rutas
# se carga la lista con las rutas
if os.path.exists(models_paths):
    print "Cargando rutas de modelos adaptados."
    paths_list = joblib.load(models_paths)
    print "Rutas cargadas en la variable 'paths_list'"
# si no
# se entrenan los modelos y se obtiene la lista con rutas
else:
    #se establecen los parametros para los modelos
    parametros = {
        'noises': [0.3, 0.5, 0.8],
        'layers': [1, 3, 5]
    }

    print "Creando modelos de adaptacion..."
    
    folder_path = os.path.join(models_path, dataset_name, tipo)
    prefix = "me1_%d_" % dims
        
    paths_list = msda_theano_pseudo_grid_search(X, parametros, folder_path, prefix)

    print "\nCreacion de modelos terminada\nGuardando rutas en %s" % models_paths
    joblib.dump(paths_list, models_paths)
    print "Rutas cargadas en la variable 'paths_list'"

Creando modelos de adaptacion...
	pr: 0.300 - l: 1
	Entrenando capa  1
	Entrenado en 0.10m

	Guardando modelo en models/amazon/msda/me1_1000_0.pkl
	pr: 0.300 - l: 3
	Entrenando capa  1
	Entrenando capa  2
	Entrenando capa  3
	Entrenado en 0.34m

	Guardando modelo en models/amazon/msda/me1_1000_1.pkl
	pr: 0.300 - l: 5
	Entrenando capa  1
	Entrenando capa  2
	Entrenando capa  3
	Entrenando capa  4
	Entrenando capa  5
	Entrenado en 0.41m

	Guardando modelo en models/amazon/msda/me1_1000_2.pkl
	pr: 0.500 - l: 1
	Entrenando capa  1
	Entrenado en 0.07m

	Guardando modelo en models/amazon/msda/me1_1000_3.pkl
	pr: 0.500 - l: 3
	Entrenando capa  1
	Entrenando capa  2
	Entrenando capa  3
	Entrenado en 0.23m

	Guardando modelo en models/amazon/msda/me1_1000_4.pkl
	pr: 0.500 - l: 5
	Entrenando capa  1
	Entrenando capa  2
	Entrenando capa  3
	Entrenando capa  4
	Entrenando capa  5
	Entrenado en 0.38m

	Guardando modelo en models/amazon/msda/me1_1000_5.pkl
	pr: 0.800 - l: 1
	Entrenando capa  1
	Entr

In [20]:
print "Modelos almacenados en:"
for ruta in paths_list:
    print "\t", ruta

Modelos almacenados en:
	models/amazon/msda/me1_1000_0.pkl
	models/amazon/msda/me1_1000_1.pkl
	models/amazon/msda/me1_1000_2.pkl
	models/amazon/msda/me1_1000_3.pkl
	models/amazon/msda/me1_1000_4.pkl
	models/amazon/msda/me1_1000_5.pkl
	models/amazon/msda/me1_1000_6.pkl
	models/amazon/msda/me1_1000_7.pkl
	models/amazon/msda/me1_1000_8.pkl


In [21]:
best_models_paths = os.path.join(models_path, dataset_name, tipo, "me1_%d_best_models.pkl" % dims)
best_models = {}

if os.path.exists(best_models_paths):
    print "Cargando rutas de los mejores modelos..."
    best_models = joblib.load(best_models_paths)
    print "Rutas cargadas"  
else:
    print "Obteniendo mejores modelos..."
    for domain in domains:
        print domain
        i = 1
        best_score = 0
        best_path = None
        best_model = None
        
        for msda_model_path in paths_list:
            
            msda_model = joblib.load(msda_model_path)
            theano_model = msda_model['model']
            pr = msda_model['pr']
            n_layers = msda_model['l']

            print "\t%d) l=%d, pr=%.3f " % (i, msda_model['l'], msda_model['pr']),

            # se obtienen los datos del dominio
            X_tr = labeled[domain]['X_tr'][:, :dims].todense()
            y_tr = np.asarray(labeled[domain]['y_tr'].todense()).argmax(axis=1)

            # se adaptan los datos
            X_deep = theano_model.predict(X_tr)

            # se obtiene el mejor score con GridSearch
            new_clf = get_best_score(X_deep, y_tr, classifier='SVC', n_jobs=4)
            new_score = new_clf.best_score_
            print "score: %.4f" % new_score
            # se guarda si es el mejor para el modelo
            if new_score > best_score:
                best_score = new_score
                best_path = msda_model_path
                best_model = msda_model
            i = i+1

        #se guarda el mejor modelo para este dominio
        print "Mejor modelo: l=%d, pr=%.3f" % (best_model['l'], best_model['pr'])
        best_models[domain] = best_path
        
    
    # se guarda el diccionario con las mejores rutas
    joblib.dump(best_models, best_models_paths)
    print "Rutas guardadas en ", best_models_paths

Obteniendo mejores modelos...
electronics
	1) l=1, pr=0.300  score: 0.8472
	2) l=3, pr=0.300  score: 0.8631
	3) l=5, pr=0.300  score: 0.8508
	4) l=1, pr=0.500  score: 0.8515
	5) l=3, pr=0.500  score: 0.8536
	6) l=5, pr=0.500  score: 0.7869
	7) l=1, pr=0.800  score: 0.8631
	8) l=3, pr=0.800  score: 0.7832
	9) l=5, pr=0.800  score: 0.5831
Mejor modelo: l=3, pr=0.300
dvd
	1) l=1, pr=0.300  score: 0.8040
	2) l=3, pr=0.300  score: 0.8342
	3) l=5, pr=0.300  score: 0.8068
	4) l=1, pr=0.500  score: 0.8155
	5) l=3, pr=0.500  score: 0.7972
	6) l=5, pr=0.500  score: 0.7032
	7) l=1, pr=0.800  score: 0.8310
	8) l=3, pr=0.800  score: 0.6810
	9) l=5, pr=0.800  score: 0.5255
Mejor modelo: l=3, pr=0.300
kitchen
	1) l=1, pr=0.300  score: 0.8620
	2) l=3, pr=0.300  score: 0.8800
	3) l=5, pr=0.300  score: 0.8649
	4) l=1, pr=0.500  score: 0.8649
	5) l=3, pr=0.500  score: 0.8641
	6) l=5, pr=0.500  score: 0.8071
	7) l=1, pr=0.800  score: 0.8741
	8) l=3, pr=0.800  score: 0.7777
	9) l=5, pr=0.800  score: 0.5354

In [22]:
for dominio, ruta in best_models.items():
    print "%s - %s" % (dominio, ruta)

dvd - models/amazon/msda/me1_1000_1.pkl
electronics - models/amazon/msda/me1_1000_1.pkl
books - models/amazon/msda/me1_1000_1.pkl
kitchen - models/amazon/msda/me1_1000_1.pkl


In [23]:
#diccionario para mantener los dominios adaptados
adapted = {}

i = 0
for domain in domains:
    best_model_path = best_models[domain]
    
    if os.path.exists(best_model_path):
        print "Cargando mejor modelo para %s" % domain
        msda_model = joblib.load(best_model_path)
        theano_model = msda_model['model']
        
        print "\tAdaptando dominio"
        X_tr = labeled[domain]['X_tr'][:, :dims].todense()

        tr_reps = theano_model.predict(X_tr)

        adapted[domain] = {
            'X_tr': tr_reps
        }
        
        i = i+1
    else:
        print "\tGenerar mejor modelo para %s" % domain
        
print "%d/%d dominios adaptados" % (i, len(domains))

Cargando mejor modelo para electronics
	Adaptando dominio
Cargando mejor modelo para dvd
	Adaptando dominio
Cargando mejor modelo para kitchen
	Adaptando dominio
Cargando mejor modelo para books
	Adaptando dominio
4/4 dominios adaptados


## Pruebas de clasificación

In [24]:
df = pd.DataFrame(columns=dataframe_columns)

i=1
tareas = len(domains)*(len(domains)-1)

# por cada par posible para adaptar
for src in domains:
    #se carga el mejor modelo para el dominio fuente
    ruta = best_models[src]
    modelo = joblib.load(ruta)
    msda_adapter = modelo['model']
    
    for tgt in domains:
        if src is not tgt:
            print "Tarea %d de %d" % (i, tareas)
            
            #baseline in-domain error
            #e_b(T,T)
            #entrenado en dominio tgt y probado en dominio tgt
            X_tr = labeled[tgt]['X_tr'][:, :dims].todense()
            y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)
            
            X_ts = labeled[tgt]['X_ts'][:, :dims].todense()
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            # se crean las rutas para cargar o crear los modelos
            model_name = "%d_%s.pkl" % (dims, tgt)
            model_path = os.path.join(models_path, dataset_name, "indomain", model_name)
            
            
            #Se realiza una clasificacion, estimando los parametros mediante cv
            svc = load_best_score(model_path, X_tr, y_tr)
            b_error = 1-svc.score(X_ts, y_ts)
            
            
            #############
            #### mSDA ###
            #############
            print "Adaptando dominios..."
            
            #entrenado en dominio src y probado en dominio tgt adaptados
            X_tr_a = adapted[src]['X_tr']
            y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)
            
            X_ts_a = msda_adapter.predict(X_ts)
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            # se calcula el transfer error
            svc_a = get_best_score(X_tr_a, y_tr, classifier='SVC', n_jobs=4)
            t_error = 1-svc_a.score(X_ts_a, y_ts)
            
            # transfer loss t
            # t_error - b_error
            t_loss = t_error - b_error
            
            tarea = src[0]+'->'+tgt[0]
            df.loc[i] = ['mSDA',tarea,src,tgt,b_error*100,t_error*100, t_loss*100]
            
            i+=1
    
print "\nPruebas completadas"

Tarea 1 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 2 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 3 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 4 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 5 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 6 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 7 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 8 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 9 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 10 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 11 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 12 de 12
Cargando modelo existente.
Adaptando dominios...

Pruebas completadas


In [25]:
df

,Adaptacion,Tarea,Fuente,Objetivo,Baseline error,Transfer error,Transfer loss
1,mSDA,e->d,electronics,dvd,16.900423,23.215580,6.315158
2,mSDA,e->k,electronics,kitchen,16.215405,12.017800,-4.197605
3,mSDA,e->b,electronics,books,23.710593,29.178229,5.467637
4,mSDA,d->e,dvd,electronics,16.397910,21.743044,5.345134
5,mSDA,d->k,dvd,kitchen,16.215405,18.722968,2.507563
6,mSDA,d->b,dvd,books,23.710593,22.043051,-1.667542
7,mSDA,k->e,kitchen,electronics,16.397910,14.612865,-1.785045
8,mSDA,k->d,kitchen,dvd,16.900423,24.140604,7.240181
9,mSDA,k->b,kitchen,books,23.710593,28.360709,4.650116
10,mSDA,b->e,books,electronics,16.397910,21.913048,5.515138


In [26]:
new_scores_path = os.path.join(scores_path,dataset_name, tipo, "me1_%d.csv" % (dims))

print "Guardando en %s" % new_scores_path
df.to_csv(new_scores_path, columns=df.columns)
print "Resultados guardados."

Guardando en scores/amazon/msda/me1_1000.csv
Resultados guardados.
